<figure>
  <img src="https://github.com/v-iashin/video_features/raw/master/docs/_assets/i3d.png" width="300" />
</figure>

The `video_features` library allows you to extract features from
raw videos in parallel with multiple GPUs.
It supports several extractors that capture visual appearance,
optical flow, and audio features. See more details in the
[GitHub repository](https://github.com/v-iashin/video_features).

See more feature extraction examples in colaboratory notebooks:
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Zd7r8uKGLGSxlil4PPnXk_4I3KOsjPpO?usp=sharing) – CLIP
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1HUlYcOJf_dArOcAaR9jaQHuM5CAZiNZc?usp=sharing) – S3D
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1LKoytZmNxtC-EuCp7pHDM6sFvK1XdwlW?usp=sharing) – I3D
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1csJgkVQ3E2qOyVlcOM-ACHGgPBBKwE2Y?usp=sharing) – R(2+1)D
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/18I95Rn1B3a2ISfD9b-o4o93m3XuHbcIY?usp=sharing) – RAFT
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/17VLdf4abQT2eoMjc6ziJ9UaRaOklTlP0?usp=sharing) – ResNet
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1r_8OnmwXKwmH0n4RxBfuICVBgpbJt_Fs?usp=sharing) – VGGish

In [4]:
!git clone https://github.com/v-iashin/video_features.git
%pip install omegaconf==2.0.6

Cloning into 'video_features'...
Updating files:  33% (60/177)
Updating files:  34% (61/177)
Updating files:  35% (62/177)
Updating files:  36% (64/177)
Updating files:  37% (66/177)
Updating files:  38% (68/177)
Updating files:  39% (70/177)
Updating files:  40% (71/177)
Updating files:  41% (73/177)
Updating files:  42% (75/177)
Updating files:  43% (77/177)
Updating files:  44% (78/177)
Updating files:  45% (80/177)
Updating files:  46% (82/177)
Updating files:  47% (84/177)
Updating files:  48% (85/177)
Updating files:  49% (87/177)
Updating files:  50% (89/177)
Updating files:  51% (91/177)
Updating files:  52% (93/177)
Updating files:  53% (94/177)
Updating files:  54% (96/177)
Updating files:  55% (98/177)
Updating files:  56% (100/177)
Updating files:  57% (101/177)
Updating files:  58% (103/177)
Updating files:  59% (105/177)
Updating files:  60% (107/177)
Updating files:  61% (108/177)
Updating files:  62% (110/177)
Updating files:  63% (112/177)
Updating files:  64% (114/177

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [12]:
%cd video_features

[WinError 2] The system cannot find the file specified: 'video_features'
e:\FYP Codes\video_features


In [11]:
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.get_device_name(0)

In [ ]:
!pip install torch
!pip install torchvision
!pip install scipy


In [21]:
import os
import numpy as np
import cv2
from tqdm import tqdm  # Import tqdm for progress tracking

# Select the feature type
feature_type = 'i3d'

# Load and patch the config
args = OmegaConf.load(build_cfg_path(feature_type))
args.device = device
args.flow_type = 'raft'  # 'pwc' is not supported on Google Colab (cupy version mismatch)

# Load the model
extractor = ExtractI3D(args)

# Directory containing video files
video_dir = '../Videos'  # Change this to your video directory

# Loop through subdirectories in the video directory
for sub_dir in os.listdir(video_dir):
    sub_dir_path = os.path.join(video_dir, sub_dir)
    
    if os.path.isdir(sub_dir_path):
        # Create a corresponding subdirectory in the extracted features folder
        output_sub_dir = os.path.join(f'../extracted_features', sub_dir)
        os.makedirs(output_sub_dir, exist_ok=True)
        # output_sub_dir = os.path.join(f'../extracted_features', sub_dir)
        # os.makedirs(output_sub_dir, exist_ok=True)
        
        # Extract features for all video files in the subdirectory
        for video_file in os.listdir(sub_dir_path):
            if video_file.endswith('.mp4'):  # Assuming your video files have the .mp4 extension
                video_path = os.path.join(sub_dir_path, video_file)
                print(f'Extracting features for {video_path}')
                
                # Get the fps of the current video
                cap = cv2.VideoCapture(video_path)
                fps = int(cap.get(cv2.CAP_PROP_FPS))
                num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                cap.release()
                
                feature_dict = extractor.extract(video_path)
                for key, value in feature_dict.items():
                    # Reshape the feature using the fps
                    reshaped_feature = np.resize(value, (2048, num_frames))
                    
                    # Save the reshaped feature in the corresponding subdirectory
                    feature_filename = os.path.join(output_sub_dir, f'{os.path.splitext(video_file)[0]}_{key}.npy')
                    np.save(feature_filename, reshaped_feature)
                    print(f'Saved {key} feature as {feature_filename}')
        
                # Update the progress bar
                print(f'Processed {video_file}')


Extracting features for ../Videos\clap\clap_1.mp4
Saved rgb feature as ../extracted_features\clap\clap_1_rgb.npy
Saved flow feature as ../extracted_features\clap\clap_1_flow.npy
Saved fps feature as ../extracted_features\clap\clap_1_fps.npy
Saved timestamps_ms feature as ../extracted_features\clap\clap_1_timestamps_ms.npy
Processed clap_1.mp4
Extracting features for ../Videos\clap\clap_10.mp4
Saved rgb feature as ../extracted_features\clap\clap_10_rgb.npy
Saved flow feature as ../extracted_features\clap\clap_10_flow.npy
Saved fps feature as ../extracted_features\clap\clap_10_fps.npy
Saved timestamps_ms feature as ../extracted_features\clap\clap_10_timestamps_ms.npy
Processed clap_10.mp4
Extracting features for ../Videos\clap\clap_100.mp4
Saved rgb feature as ../extracted_features\clap\clap_100_rgb.npy
Saved flow feature as ../extracted_features\clap\clap_100_flow.npy
Saved fps feature as ../extracted_features\clap\clap_100_fps.npy
Saved timestamps_ms feature as ../extracted_features\cl

In [ ]:
!pip freeze